# DSPM-DevSecOps Evidence Pipeline — Quickstart

This notebook validates the **serverless evidence pipeline** end-to-end:

- install dependencies
- run the CLI locally
- generate an evidence bundle (manifest + receipts)
- verify integrity via hashes

> Colab tip: Run cells top-to-bottom.

## GDrive Setup

In [1]:
import os, pathlib, subprocess
from  google.colab import drive
if not os.path.exists("/content/drive"):
    drive.mount('/content/drive')
else:
  print("Drive already mounted")

search_item = '01_Quickstart_Evidence_Pipeline.ipynb'

l_result_labels = subprocess.run(["find","/content/drive/MyDrive","-type","f","-iname",f"*{search_item}*"],capture_output=True, text=True)
print(f"Labels_proj.csv path: {l_result_labels.stdout.splitlines()}")



Mounted at /content/drive
Labels_proj.csv path: ['/content/drive/MyDrive/DSPM-DevSecOps-EphemeralCompute-EvidencePipeline/notebooks/01_Quickstart_Evidence_Pipeline.ipynb']


In [2]:
%cd /content/drive/MyDrive/DSPM-DevSecOps-EphemeralCompute-EvidencePipeline

!python -V
!pip -q install -r requirements.txt
!pip -q install -e .


/content/drive/MyDrive/DSPM-DevSecOps-EphemeralCompute-EvidencePipeline
Python 3.12.12
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for dspm-devsecops (pyproject.toml) ... done


## 1) Smoke test the CLI

In [3]:
!python -m dspm_devsecops.cli --help

usage: dspm-devsecops [-h] [--repo-root REPO_ROOT] {run,demo,ci} ...

positional arguments:
  {run,demo,ci}
    run                 Run the full pipeline (default)
    demo                Run pipeline writing outputs to --out (safe local
                        demo)
    ci                  Run pipeline writing outputs to --out (CI simulation)

options:
  -h, --help            show this help message and exit
  --repo-root REPO_ROOT
                        Path to repo root (contains examples/ etc.)


## 2) Generate a sample evidence bundle

This creates deterministic synthetic artifacts under `./out/` (safe to commit/attach).

In [4]:
import os, json, pathlib, shutil
out = pathlib.Path("_out")
if out.exists():
    shutil.rmtree(out)
out.mkdir(parents=True, exist_ok=True)

%cd /content/drive/MyDrive/00_Final_3/DSPM_DevSecOps_Post3_Serverless_EvidencePipeline_v1_1/DSPM-DevSecOps-EphemeralCompute-EvidencePipeline




# Run a sample scan + evidence generation (local demo mode)
!python -m dspm_devsecops.cli --repo-root . demo --out _out

[Errno 2] No such file or directory: '/content/drive/MyDrive/00_Final_3/DSPM_DevSecOps_Post3_Serverless_EvidencePipeline_v1_1/DSPM-DevSecOps-EphemeralCompute-EvidencePipeline'
/content/drive/MyDrive/DSPM-DevSecOps-EphemeralCompute-EvidencePipeline
                    DSPM + DevSecOps Pipeline Summary (v1.1)                    
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Category            ┃ Count ┃ Notes                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Terraform findings  │     2 │ IaC posture + network/iam/serverless patterns  │
│ Serverless findings │     7 │ Triggers, env leakage, VPC attachment, logging │
│ Base Risk (0-100)   │    60 │ From IaC findings (demo)                       │
│ Policy Gate         │  FAIL │ Policy DSL evaluated against normalized assets │
│ Artifacts           │     - │ Wrote outputs to                               │
│                     │

## 3) Inspect outputs

In [5]:
import pathlib, json
out = pathlib.Path("_out")
for p in sorted(out.rglob("*")):
    print(p.as_posix())

_out/destroy_closure.json
_out/evidence
_out/evidence/manifest.sha256.json
_out/evidence/receipt_audit.json
_out/evidence/receipt_create.json
_out/evidence/receipt_destroy.json
_out/evidence/receipt_maintain.json
_out/gate_status.json
_out/normalized_assets.json
_out/policy_results.json
_out/risk_score_base.json
_out/scans
_out/scans/serverless_findings.json
_out/scans/terraform_findings.json
_out/trigger_graph.json


## 4) Verify the evidence manifest hashes

This ensures the evidence bundle is *tamper-evident*.

In [6]:
!ls -la out || true
!ls -la out/evidence || true
!find out -maxdepth 3 -type f -print


total 19
-rw------- 1 root root  203 Feb 15 21:06 destroy_closure.json
drwx------ 2 root root 4096 Feb 15 21:06 evidence
-rw------- 1 root root   22 Feb 15 21:06 gate_status.json
-rw------- 1 root root 2944 Feb 15 21:06 normalized_assets.json
-rw------- 1 root root 5945 Feb 15 21:06 policy_results.json
-rw------- 1 root root  107 Feb 15 21:06 risk_score_base.json
drwx------ 2 root root 4096 Feb 15 21:06 scans
-rw------- 1 root root  499 Feb 15 21:06 trigger_graph.json
total 4
-rw------- 1 root root 1697 Feb 15 21:06 manifest.sha256.json
-rw------- 1 root root  140 Feb 15 21:06 receipt_audit.json
-rw------- 1 root root  220 Feb 15 21:06 receipt_create.json
-rw------- 1 root root  343 Feb 15 21:06 receipt_destroy.json
-rw------- 1 root root  289 Feb 15 21:06 receipt_maintain.json
out/evidence/manifest.sha256.json
out/evidence/receipt_audit.json
out/evidence/receipt_create.json
out/evidence/receipt_destroy.json
out/evidence/receipt_maintain.json
out/risk_score_base.json
out/scans/terrafor

In [7]:
import hashlib, json, pathlib
import os

manifest_path = pathlib.Path("out/evidence/manifest.sha256.json")
manifest = json.loads(manifest_path.read_text())

def sha256_file(path: pathlib.Path) -> str:
    h = hashlib.sha256()
    with path.open("rb") as f:
        for chunk in iter(lambda: f.read(1024*1024), b""):
            h.update(chunk)
    return h.hexdigest()

bad = []

for item in manifest["entries"]:
    full_path_in_manifest = item["path"]
    expected = item["sha256"]

    # Extract the relative path assuming it starts with 'out/'
    # This handles the case where manifest paths are absolute paths from a temporary location.
    try:
        out_start_index = full_path_in_manifest.index('out/')
        relative_path_to_check = pathlib.Path(full_path_in_manifest[out_start_index:])
    except ValueError:
        print(f"Error: 'out/' not found in manifest path for {full_path_in_manifest}. Skipping verification.")
        bad.append((full_path_in_manifest, expected, "ERROR: Path format unexpected"))
        continue

    # Use the extracted relative path directly, as it is relative to the current working directory
    actual = sha256_file(relative_path_to_check)

    if actual != expected:
        bad.append((str(relative_path_to_check), expected, actual))

print("Items:", len(manifest["entries"]))
print("Verification:", "PASS" if not bad else "FAIL")
bad[:5]

Items: 7
Verification: PASS


[]

## 5) What we validated

- The pipeline produced artifacts
- A manifest enumerates artifact hashes
- Hash verification passes (integrity)